In [1]:
import pandas as pd
import os
import pymongo as pm
import re
pd.options.display.max_rows = 200

In [2]:
workbook_path = os.path.join('DFS', 'Resources', 'DraftKingsNoNotes.xlsx')
workbook = pd.ExcelFile(workbook_path)


In [3]:
vegas_dash = workbook.parse('Vegas Dash')
top_guns = workbook.parse('TOP GUNS')
wr_dash = workbook.parse('WR Dash')
te_dash = workbook.parse('TE Dash')
rb_dash = workbook.parse('RB Dash')
qb_dash = workbook.parse('QB Dash')
dst_dash = workbook.parse('DST Dash')

In [4]:
client = pm.MongoClient('localhost', 27017)

db = client.test_database
SS_Data = db['Saber_Sim_Data']
_4f4_Proj = db['_4for4_Projection_Data']
_4f4_Ceil = db['_4for4_Ceiling_Data']
_4f4_WR_cb = db['_4for4_WR_cb']
_4f4_RedZ = db['_4for4__RedZone_Data']
_4f4_WR_fp = db['_4for4_WR_fantasy_points']
_4f4_RB_fp = db['_4for4_RB_fantasy_points']
_4f4_RB_tar = db['_4for4_RB_target']
airy_WR = db['AirY_WR']
airy_TE = db['AirY_TE']
ETR_Ceil = db['ETR_Ceiling']

In [5]:
def position_names(pos, db):
    data = []

    for collection in db.list_collection_names():
        col = db[collection]
        positions = ['Pos', 'Position', 'position']
        for fil in positions:
            doc_count = col.count_documents(filter={fil : pos})
            if doc_count > 0:
                cursor = col.find({fil : pos},{'_id':False})
                for i in range(doc_count):
                    data.append(cursor[i])
    names = []
    for row in data:
        if 'full_name' in row:
            name = row['full_name']
            if not any(word in name for word in names):
                names.append(name)
        elif 'Name' in row:
            name = row['Name']
            if not any(word in name for word in names):
                names.append(name)
        elif 'Player' in row:
            name = row['Player']
            if not any(word in name for word in names):
                names.append(name)
    return names

In [134]:
def stack_app_query_position(pos,SS_Data):
    data = []
    doc_count = SS_Data.count_documents(filter=({'Position' : pos}))
    cursor = SS_Data.find({'Position' : pos}, {'_id':False, 'Name': True, 'Price':True, 'Projection':True})
    for i in range(doc_count):
        data.append(cursor[i])
                    
    return data


In [145]:
qb_stack = {x: stack_app_query_position(x, SS_Data) for x in ['QB', 'WR', 'TE', 'RB', 'DST']}

In [146]:
qb_stack

{'QB': [{'Name': 'Patrick Mahomes', 'Projection': 24.7241, 'Price': 7200},
  {'Name': 'Aaron Rodgers', 'Projection': 23.2646, 'Price': 6900},
  {'Name': 'Dak Prescott', 'Projection': 21.8913, 'Price': 6300},
  {'Name': 'Tom Brady', 'Projection': 21.6051, 'Price': 6000},
  {'Name': 'Matt Ryan', 'Projection': 20.3802, 'Price': 6500},
  {'Name': 'Ryan Tannehill', 'Projection': 20.3171, 'Price': 6800},
  {'Name': 'Jameis Winston', 'Projection': 20.2265, 'Price': 6600},
  {'Name': 'Carson Wentz', 'Projection': 19.8723, 'Price': 6100},
  {'Name': 'Drew Brees', 'Projection': 19.7272, 'Price': 7000},
  {'Name': 'Mitchell Trubisky', 'Projection': 17.4971, 'Price': 5700},
  {'Name': 'Jacoby Brissett', 'Projection': 17.2627, 'Price': 5800},
  {'Name': 'Baker Mayfield', 'Projection': 16.9483, 'Price': 5700},
  {'Name': 'Daniel Jones', 'Projection': 16.9253, 'Price': 6400},
  {'Name': 'Robert Griffin III', 'Projection': 16.8834, 'Price': 5100},
  {'Name': 'Philip Rivers', 'Projection': 16.4248, 'Pr

In [147]:
pd.DataFrame(qb_stack['TE'])

,Name,Projection,Price
0,Travis Kelce,20.01110,7000
1,Dallas Goedert,17.20170,4900
2,OJ Howard,13.83720,4300
3,Austin Hooper,13.41770,5800
4,Tyler Higbee,13.08770,5600
5,Darren Waller,13.05550,5400
6,Cameron Brate,11.03060,3200
7,Jack Doyle,10.21980,4100
8,Jared Cook,9.87720,5200
9,Hunter Henry,9.40455,4500
